In [1]:
import emoji

In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [4]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [5]:
from keras.utils import to_categorical
xtrain = train[0]

xtest = test[0]
xtrain.shape

Using TensorFlow backend.


(132,)

2

In [8]:
for ix in range(10):
    print(xtrain[ix],emoji.emojize(emoji_dictionary[str(train[1][ix])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [9]:
ytrain = to_categorical(train[1],num_classes=5)
ytest = to_categorical(test[1],num_classes=5)

In [10]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs
f.close()

In [11]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [12]:
t = getOutputEmbeddings(xtrain)
te = getOutputEmbeddings(xtest)

c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [13]:
type(xtrain)

pandas.core.series.Series

In [14]:
from keras.models import Sequential
from keras.layers import *

In [15]:
model = Sequential()
model.add(LSTM(64,input_shape = (10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

W0804 15:59:07.130746  3496 deprecation_wrapper.py:119] From c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 15:59:07.181625  3496 deprecation_wrapper.py:119] From c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 15:59:07.184614  3496 deprecation_wrapper.py:119] From c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0804 15:59:08.003491  3496 deprecation_wrapper.py:119] From c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [16]:

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("bestmodel.h5",monitor = "val_loss",verbose = True,save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_acc',patience = 10)
hist = model.fit(t,ytrain,epochs = 20,shuffle=True,validation_split=0.2,batch_size=64)

W0804 15:59:08.998108  3496 deprecation.py:323] From c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 105 samples, validate on 27 samples
Epoch 1/20
105/105 [==============================] - 4s 40ms/step - loss: 1.6045 - acc: 0.2381 - val_loss: 1.6013 - val_acc: 0.2963
Epoch 2/20
105/105 [==============================] - 0s 420us/step - loss: 1.5801 - acc: 0.2857 - val_loss: 1.5955 - val_acc: 0.2593
Epoch 3/20
105/105 [==============================] - 0s 323us/step - loss: 1.5494 - acc: 0.3905 - val_loss: 1.5926 - val_acc: 0.2593
Epoch 4/20
105/105 [==============================] - 0s 266us/step - loss: 1.5218 - acc: 0.4095 - val_loss: 1.5925 - val_acc: 0.2593
Epoch 5/20
105/105 [==============================] - 0s 237us/step - loss: 1.5041 - acc: 0.3714 - val_loss: 1.5954 - val_acc: 0.2222
Epoch 6/20
105/105 [==============================] - 0s 266us/step - loss: 1.4933 - acc: 0.4190 - val_loss: 1.5992 - val_acc: 0.2222
Epoch 7/20
105/105 [==============================] - 0s 218us/step - loss: 1.4618 - acc: 0.4381 - val_loss: 1.6045 - val_acc: 0.2222
Epoch 8/20
105/105

In [17]:
y_  = model.predict_classes(te)
y_

array([3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 4, 3, 3,
       2, 2, 3, 3, 2, 2, 3, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 1,
       2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 2, 3], dtype=int64)

In [18]:
for ix in range(30):
    print(xtest[ix])
    print(emoji.emojize(emoji_dictionary[str(np.argmax(ytest[ix]))]))
    print(emoji.emojize(emoji_dictionary[str(y_[ix])]))
    

['I', 'want', 'to', 'eat']
🍴
😓
['he', 'did', 'not', 'answer']
😓
😓
['he', 'got', 'a', 'raise']
😁
😓
['she', 'got', 'me', 'a', 'present']
❤️
😁
['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny']
😁
😁
['he', 'is', 'a', 'good', 'friend']
❤️
😁
['I', 'am', 'upset']
❤️
😓
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight']
❤️
😁
['where', 'is', 'the', 'food']
🍴
😁
['Stop', 'making', 'this', 'joke', 'ha', 'ha', 'ha']
😁
😁
['where', 'is', 'the', 'ball']
⚾
😁
['work', 'is', 'hard']
😓
😁
['This', 'girl', 'is', 'messing', 'with', 'me']
😓
😁
['are', 'you', 'serious', 'ha', 'ha']
😁
😁
['Let', 'us', 'go', 'play', 'baseball']
⚾
⚾
['This', 'stupid', 'grader', 'is', 'not', 'working']
😓
😁
['work', 'is', 'horrible']
😓
😁
['Congratulation', 'for', 'having', 'a', 'baby']
😁
😁
['stop', 'messing', 'around']
😓
😓
['any', 'suggestions', 'for', 'dinner']
🍴
🍴
['I', 'love', 'taking', 'breaks']
❤️
😓
['you', 'brighten', 'my', 'day']
😁
😓
['I', 'boiled', 'rice']
🍴
😁
['she', 'is', 'a', 'bully']
😓
😁
['Why', 'are', 'you', 'feeling',

In [27]:

model.evaluate(te,ytest)

56/56 [==============================] - 0s 199us/step


[1.3838989394051688, 0.5714285629136222]